In [31]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm, tqdm_notebook

import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD, lr_scheduler
import shutil, sys   
import copy
import torch.nn as nn

import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy 

from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader



In [32]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()

In [33]:
def test_performance(model, test_loader):
        model.eval()
        running_loss = 0
        running_corrects = 0

        with torch.no_grad():
            
            number_test_steps = 0
            for batch_id, (inputs, labels) in enumerate(test_loader):
                inputs, labels = Variable(inputs.to(device)), Variable(labels.to(device))
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item() # sum up loss of all minibatches
                number_test_steps += 1
                
                _, preds = torch.max(outputs, 1)
                running_corrects += torch.sum(preds == labels.data).cpu()
                
            
            epoch_testloss = running_loss / number_test_steps
            epoch_testaccuracy = running_corrects.double() / len(test_loader.dataset)
            
            print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            epoch_testloss, running_corrects, len(test_loader.dataset),
            100. * epoch_testaccuracy))

# Evaluating models on Actor wise split data

In [34]:
# Change to fit hardware

data_dir = '/scratch/kv942/ccm_proj/datadir_act/'
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
test_loader = DataLoader(image_datasets['test'],
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                       )


## 1) Alexnet

In [35]:
PATH = '/scratch/sg5783/CCM_Project/final_folder/model_weights/actor_wise_split/alexnet_40.pth'
model = torchvision.models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 8)

model.load_state_dict(torch.load(PATH, map_location='cpu'))
test_performance(model, test_loader)

Test set: Average loss: 2.4848, Accuracy: 2220/4450 (49.89%)



## 2) VGG16

In [38]:
PATH = '/scratch/sg5783/CCM_Project/final_folder/model_weights/actor_wise_split/VGG16_40.pth'
model = torchvision.models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 8)

model.load_state_dict(torch.load(PATH, map_location='cpu'))
test_performance(model, test_loader)

Test set: Average loss: 1.7952, Accuracy: 2359/4450 (53.01%)



# Evaluating models on Random split data

In [39]:
# Change to fit hardware

data_dir = '/scratch/kv942/ccm_proj/datadir_rnd/'
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
test_loader = DataLoader(image_datasets['test'],
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                       )


## 1) Alexnet

In [40]:
PATH = '/scratch/sg5783/CCM_Project/final_folder/model_weights/random_data_split/alexnet_40.pth'
model = torchvision.models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 8)

model.load_state_dict(torch.load(PATH, map_location='cpu'))
test_performance(model, test_loader)

Test set: Average loss: 0.0930, Accuracy: 4693/4837 (97.02%)



## 2) VGG16

In [41]:
PATH = '/scratch/sg5783/CCM_Project/final_folder/model_weights/random_data_split/VGG16_40.pth'
model = torchvision.models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 8)

model.load_state_dict(torch.load(PATH, map_location='cpu'))
test_performance(model, test_loader)

Test set: Average loss: 0.0739, Accuracy: 4739/4837 (97.97%)

